<a href="https://colab.research.google.com/github/lidar532/JSON2C/blob/main/JSON2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert JSON string to a "C" sprintf formatting string.
Enter or paste a JSON string below that you would like to conver to a "C" sprintf formatting
statement for use in an Arduino or other project.  The JSON will be converted to a
compilable "C" language formatting string you can copy to your Arduino code.  You can use [JASONata](https://try.jsonata.org/) or [JSON Editor Online](https://jsoneditoronline.org/) or other JSON tool to build, verify, and test your JSON string. Simply use "C" formatting specifiers where your values go, and js2C will generate the "C" formatting string for you.  Note that you can enter a "C" comment 
by using the `"comment_"` JSON key and placing your comment text as the value string.  You can name your "C" format variable with the JSON key `"format_name_"` with the desired "C" name in the value string.  See the example below.  I enclosed my JSON in triple quotes `'''` but it should work ok with single quotes as well.  After you enter your JSON string, press the Run arrow button to load it into the js variable. 

In [ ]:
#@title Step 1. Put your JSON here. { display-mode: "both" }
js = '''
{ 
  "format_name_":"my_name",
  "comment_": " This is a comment. It can contain numbers and any of the following chars {}[],.;?<>| %^&*()@#$ : %!",
  "myJSON": {
    "dest": "%s",
    "sss": "%0.2f",
    "flash": [
      "%f",
      "%4.2lf",
      "%ld",
      "%lu",
      "%u",
      "%d",
      "%c"
    ]
  }
}
'''

In [ ]:
#@title Step 2. Click the Run button to do the conversion.  { display-mode: "form" }
#@markdown Your orginal JSON will be encapsulated in 
#@markdown within a "C" ```/* comment */``` block and
#@markdown the "C" format string below.  Copy and paste the resulting 
#@markdown comments and code into your project.
import re
import os

def js2c( js, comment="" ):
  '''
  j2sc( js).  Converts a JSON message string that contains "C" language
  format specifiers into an actual "C" formatting string for use with
  sprintf or printf.  The purpose is to allow one to build and test
  a JSON string using JSONata, or other JSON tool, and then automatically
  convert it to a "C" formatting string.

  Regex expressions developed using: https://regex101.com/
  Testing JSON strings was done with: https://try.jsonata.org/
  "C" verification was done using: https://www.onlinegdb.com/#

  '''
# Extract the variable definition name, if any,
# leaving it in var_def, and the updated js with the
# var_def removed in js2

  regex = "(^\s+\"comment_\"\s*:\s*)(\")([\sa-zA-Z\.\d%!@#$%^&\*\(\)\[\]\{\}[<\>;:|,\?]+)(\")\s*,"
  m = re.search(regex, js, re.MULTILINE )
  #print(m)
  if m:
    js1 = js[0:m.start()]+js[m.end():]
    comment = f'{m.groups()[2]}'
    js2 = js1[0:m.start()]+js1[m.end():]
    #print(f'/*\n {comment}\n/*\n {js1}')
  else:
    comment = ""
    js2 = js

  regex = "[ \t]*\"format_name_\"\ *: *(\")(.*)(\"),[\s\n]*"
  m = re.search(regex, js1, re.MULTILINE )
  if m:
    js2 = js1[0:m.start()]+js1[m.end():]
    var_def = f'const char {m.groups()[1]}[] = ""'
  else:
    var_def = f'const char var_def[] = ""'
    js2 = js1
  #return js2

  regex = r"(\")([%][\d]*\.*[\d]*l?f)(\")"          # %f %lf
  subst = "\\2"
  js3 = re.sub(regex, subst, js2, 0, re.MULTILINE)
  #return js4

  regex = r"(\")([%][\d]*[ul]?[ldu])(\")"           # %u %d %ul %ld
  subst = "\\2"
  js4 = re.sub(regex, subst, js3, 0, re.MULTILINE)
  #return js4

  regex = r"(\")([%][\d]*[sc])(\")"                 # %c, %s or %10s
  subst = "~~~\\2~~~"                               # mark strings with ~~~
  js5 = re.sub(regex, subst, js4, 0, re.MULTILINE)
  #return js5

  js6 = js5.replace('"','\\"')                      # Now escape all quotes
  #return js6

  js7 = js6.replace('~~~','\\"')                    # Now, repace the ~~~ with escaled quotes
  #return js7

  regex = r"(\n\s*\n)"                              # remove blank lines
  subst = ""
  js8 = re.sub(regex, subst, js7, 0, re.MULTILINE)
  #return js8

  regex = r"(  +)"                                  # remove excess spaces.
  subst = ""
  js9 = re.sub(regex, subst, js8, 0, re.MULTILINE)

  js10 = js9.replace('\n', '\"\n\"')                 # Put quotes at the beg and end of all lines.
  #return js10

  js11 = f'"{js10}"'                                 # add to beg and end of the string
  #return js11

  js12 = f'{var_def}\n{js11};'
  #return js12

  doc = "/* This was autogenerated from the above JSON using js2c.ipynb */"
  js13 = f'/* {comment} {js} */\n\n{doc}\n{js12}\n\n'
  return js13

jsc = js2c(js)
print(jsc)

In [ ]:
#@title Step 3. Generate a test.c program that will output your JSON string. { display-mode: "form" }
tester = {"name":"test1",
"src"  : f'''
#include <stdio.h>

{jsc}

int main()
{{
    char                s[] ="a string";
    float               a   = 1.0;
    double              b   = 2.0;
    float               c   = 3.0;
    long int            d   = 4;
    unsigned long int   e   = 5;
    unsigned int        f   = 6;
    int                 g   = 7;
    char                h   = 'C';
    printf("\\nYour serialized JSON string is below. \\n"
    "Copy and paste to https://try.jsonata.org/ to verify it.\\n\\n");
    printf( my_name, s, a, b, c, d, e, f, g, h );
    return 0;
}}
'''
}

if os.path.exists( '/usr/bin/clang-format') == 0:     # Install clang-format if not present
  ! apt-get install clang-format

tester['name.c'] = f"{tester['name']}.c"
tester['output'] = f"{tester['name']}.json"
with open(tester['name.c'], 'w') as file:
    file.write(tester['src'])

! clang-format {tester['name.c']}

In [ ]:
#@title Step 4. Compile the test program with `gcc`, run it and cat the JSON output  
# Test with gcc.
if os.path.exists( '/usr/bin/clang-format') == 0:     # Install clang-format if not present
  ! apt-get install clang-format
! gcc {tester['name.c']} -o {tester['name']}          # compile the test program.
! ./{tester['name']} > {tester['output']}             # now run it.
#! clang-format {tester['output']}                     # Pretty print the resulting JSON.
! cat {tester['output']}